In [1]:
import pandas as pd
from pykrx import stock
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 함수 정의

## 지정한 날짜가 영업일이 아닐 때 다음 영업일을 찾아 주는 함수

In [2]:
def get_next_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 다음날로 이동
        dt = datetime.strptime(date, "%Y%m%d") + timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 다음 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

## 지정한 날짜가 영업일이 아닐 때 이전 영업일을 찾아 주는 함수

In [3]:
def get_previous_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 이전날로 이동
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 이전 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

## 지정한 기간동안의 코스피200의 종목 리스트를 반환하는 함수

In [4]:
def kospi200_list(start_year, end_year):
    # 결과를 저장할 리스트 생성
    records = []

    for year in range(start_year-1, end_year + 1):
        date = f"{year}0430"  # 4월말 기준 (휴장일일 경우 이후 영업입)
        trading_date = get_previous_trading_day(date)
        df = stock.get_index_portfolio_deposit_file("1028", trading_date, alternative = True)
        # 각 코드에 대해 연도와 함께 리스트에 저장
        for code in df:
            records.append({'연도': year, '주식코드': code})

    # 리스트를 데이터프레임으로 변환
    result_df = pd.DataFrame(records)

    frequency = result_df['주식코드'].value_counts()
    frequency_df = frequency.reset_index()
    frequency_df.columns = ['주식코드', '빈도수']
    code_df = frequency_df[['주식코드']]
    code_list = code_df['주식코드'].tolist()

    return code_list

# 기간 내 코스피 200 종목 리스트 수집

In [6]:
# 코스피200 종목 리스트 수집
ticker_list = kospi200_list(2015, 2025)

# 누락된 2014년 코스피200 종목 데이터 수집 후 병합
list_df = pd.read_csv("data/2014_04_kospi200.csv",dtype={"종목코드" : str}, encoding='cp949')
list_df["날짜"] = "2014-04-30"
list_df["연도"] = "2014"
list_df = list_df[["종목코드","날짜","연도","상장시가총액"]]
list_df.rename(columns={'상장시가총액' : '시가총액', '종목코드' : '주식코드'}, inplace=True)
code_list = list_df["주식코드"].to_list()
ticker_list = ticker_list+code_list
ticker_list = list(set(ticker_list))

KRX web server does NOT provide data prior to 2014/05/01.
20160430는 휴장일입니다. 이전 영업일을 검색합니다...
20170430는 휴장일입니다. 이전 영업일을 검색합니다...
20170429는 휴장일입니다. 이전 영업일을 검색합니다...
20200430는 휴장일입니다. 이전 영업일을 검색합니다...
20220430는 휴장일입니다. 이전 영업일을 검색합니다...
20230430는 휴장일입니다. 이전 영업일을 검색합니다...
20230429는 휴장일입니다. 이전 영업일을 검색합니다...


FileNotFoundError: [Errno 2] No such file or directory: 'data/2014_04_kospi200.csv'

# 수익률 수집

In [ ]:
start_year = 2015
end_year = 2025

records = []
for year in range(start_year-1, end_year+1):
    start_date = f"{year}0430"
    trading_date = get_previous_trading_day(start_date)
    for code in ticker_list:
        cap_df = stock.get_market_ohlcv(trading_date, trading_date, code, adjusted=True)
        if trading_date in cap_df.index:
            close = cap_df.loc[trading_date, '종가']
        # open = cap_df.loc[code, '시가']
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': close,
                # '시가': open
            })
        else:
            records.append({
                '주식코드': code,
                '연도': year,
                '날짜': trading_date,
                '종가': None,
                # '시가': None
            })

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(records)
df.sort_values(by=['주식코드','연도'],ascending=True, ignore_index=True, inplace=True)

df['수익률'] = df.groupby('주식코드')['종가'].pct_change(1).shift(-1)

df = df.replace([np.inf, -np.inf], np.nan)

df = df[df['수익률'] != -1]

df = df[['주식코드','연도','종가','수익률']]
df.dropna(inplace=True)

In [187]:
df.to_excel("data/수익률.xlsx", index=False)

# 수익률과 재무지표와 합치기

In [214]:
industry_df = pd.read_excel("data/final_data.xlsx", dtype={'거래소코드' : str})

rtn_df = pd.read_excel("data/수익률.xlsx", dtype={'주식코드' : str})

In [215]:
def ts_data_loader(df):
    ts_list = df.columns.unique()

    ts_df = df.copy()
    ts_df['회계년도'] = ts_df['회계년도'].str.split('/').str[0].astype(int) + 1
    ts_df['거래소코드'] = ts_df['거래소코드'].str.strip()
    ts_df.rename(columns = {'회계년도':'연도',
                            '거래소코드':'주식코드'}, inplace=True)
    ts_df = ts_df.dropna()

    # 중복 확인
    duplicates = ts_df.duplicated(subset=['주식코드', '연도'], keep=False)
    ts_df[duplicates]

    # 중복된 ('주식코드', '연도') 조합에서 첫 번째 값만 남김
    ts_df = ts_df.drop_duplicates(subset=['주식코드', '연도'], keep='first').sort_values(by=['주식코드','연도'])

    return ts_df

In [216]:
industry_data = ts_data_loader(industry_df)

In [217]:
final_industry_data = pd.merge(industry_data, rtn_df, on=['주식코드','연도'], how='inner')
final_industry_data_rtn = final_industry_data.replace([np.inf, -np.inf], np.nan)

In [218]:
# (원)단위 컬럼 제거
col_list = []
for col in final_industry_data_rtn.columns:
    if '원)' in col:
        col_list.append(col)
# print(col_list)
final_industry_data_rtn.drop(columns=col_list,inplace=True)

In [219]:
final_industry_data_rtn

,회사명,주식코드,연도,총자본증가율(IFRS),유형자산증가율(IFRS),비유동생물자산증가율(IFRS),투자부동산증가율(IFRS),비유동자산증가율(IFRS),유동자산증가율(IFRS),재고자산증가율(IFRS),...,자본집약도(IFRS),총자본투자효율(IFRS),설비투자효율(IFRS),기계투자효율(IFRS),부가가치율(IFRS),노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),종가,수익률
0,(주)경방,000050,2016,-1.22,-4.16,0.0,-1.29,-1.31,0.11,8.61,...,2047.99,8.57,41.80,379.17,35.08,19.66,80.34,20.01,17400,-0.155172
1,(주)경방,000050,2017,-1.79,-3.70,0.0,-1.30,-1.64,-3.85,-10.48,...,0.00,8.89,44.22,502.67,35.04,19.01,80.99,28.16,14700,0.013605
2,(주)경방,000050,2018,-3.07,-7.75,0.0,-1.26,-2.50,-11.26,-33.22,...,0.00,9.12,47.66,647.91,37.96,18.25,81.75,23.48,14900,-0.248322
3,(주)경방,000050,2019,2.29,-6.58,0.0,-1.13,0.96,23.35,58.28,...,0.00,7.22,41.34,1552.83,31.75,19.89,80.11,32.96,11200,-0.107143
4,(주)경방,000050,2020,5.68,-69.74,0.0,5.71,-5.79,153.86,-60.76,...,0.00,6.22,134.74,18933.42,31.82,15.85,84.15,27.55,10000,0.315000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,(주)에프앤에프,383220,2023,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,46.89,920.61,0.00,38.57,7.43,92.57,61.88,141000,-0.532624
2387,(주)에프앤에프,383220,2024,32.54,82.49,0.0,0.00,34.26,29.85,9.24,...,0.00,36.91,907.64,0.00,38.39,8.78,91.22,63.87,65900,0.071320
2388,에스케이스퀘어(주),402340,2023,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,6.34,2127.64,80447.21,80.42,9.21,90.79,91.10,42450,0.802120
2389,에스케이스퀘어(주),402340,2024,-1.77,-1.78,0.0,0.00,-5.59,111.96,0.00,...,77491.44,-2.38,-694.84,-33101.32,-98.60,0.00,0.00,0.00,76500,0.162092


In [201]:
final_industry_data_rtn = final_industry_data_rtn.drop(columns = ["종가"])

In [203]:
final_industry_data_rtn.to_excel("data/제조업_수익률.xlsx", index=False)

# 코스피 200 연간 수익률(벤치마크)

In [41]:
def get_previous_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 이전날로 이동
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 이전 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

start_year = 2014
end_year = 2024

records = []
for year in range(start_year-1, end_year + 1):
    date = f"{year}1231"  # 4월말 기준 (휴장일일 경우 이후 영업입)
    trading_date = get_previous_trading_day(date)
    df = stock.get_index_ohlcv(trading_date, trading_date, "1001")
    df = df.reset_index(drop=True)
    df['회계년도'] = f"{year}/12"
    records.append(df)

# DataFrame 합치기
final_df = pd.concat(records, ignore_index=True)

20131231는 휴장일입니다. 이전 영업일을 검색합니다...
20141231는 휴장일입니다. 이전 영업일을 검색합니다...
20151231는 휴장일입니다. 이전 영업일을 검색합니다...
20161231는 휴장일입니다. 이전 영업일을 검색합니다...
20161230는 휴장일입니다. 이전 영업일을 검색합니다...
20171231는 휴장일입니다. 이전 영업일을 검색합니다...
20171230는 휴장일입니다. 이전 영업일을 검색합니다...
20171229는 휴장일입니다. 이전 영업일을 검색합니다...
20181231는 휴장일입니다. 이전 영업일을 검색합니다...
20181230는 휴장일입니다. 이전 영업일을 검색합니다...
20181229는 휴장일입니다. 이전 영업일을 검색합니다...
20191231는 휴장일입니다. 이전 영업일을 검색합니다...
20201231는 휴장일입니다. 이전 영업일을 검색합니다...
20211231는 휴장일입니다. 이전 영업일을 검색합니다...
20221231는 휴장일입니다. 이전 영업일을 검색합니다...
20221230는 휴장일입니다. 이전 영업일을 검색합니다...
20231231는 휴장일입니다. 이전 영업일을 검색합니다...
20231230는 휴장일입니다. 이전 영업일을 검색합니다...
20231229는 휴장일입니다. 이전 영업일을 검색합니다...
20241231는 휴장일입니다. 이전 영업일을 검색합니다...


In [42]:
final_df

코스피,시가,고가,저가,종가,거래량,거래대금,상장시가총액,회계년도
0,2004.32,2011.34,1998.61,2011.34,172231154,3261548101137,1185973723790511,2013/12
1,1934.51,1937.55,1910.78,1915.59,256436661,3695016109806,1192252867277006,2014/12
2,1973.03,1974.67,1958.78,1961.31,304462557,3706627074039,1242832089300653,2015/12
3,2020.42,2026.57,2016.25,2026.46,264521833,3009199072166,1308440373892236,2016/12
4,2439.31,2467.51,2437.26,2467.49,220336303,4274352132703,1605820911910270,2017/12
5,2036.70,2046.97,2035.41,2041.04,352677713,4120695824217,1343971857985694,2018/12
6,2202.62,2208.04,2195.65,2197.67,416403484,4113597620249,1475909365805673,2019/12
7,2820.36,2878.21,2809.35,2873.47,1074033495,17928854517044,1980543161609061,2020/12
8,2999.75,3005.36,2975.74,2977.65,461038202,8727458062998,2203366545861792,2021/12
9,2265.73,2272.67,2236.38,2236.40,361194425,6036542050335,1767235220931085,2022/12


In [43]:
df1 = pd.read_excel('/Users/kimboyun/Desktop/Code/UBION_Project2/코스피12.xlsx')
df1['회계년도'] = '2012/12'
df1

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,상장시가총액,회계년도
0,2012/12/28,1997.05,9.7,0.49,1984.84,2002.67,1984.84,302927,3299123,1154294167,2012/12


In [44]:
kospi_final_df  = pd.concat([final_df,df1] , join='outer')

In [45]:
close_df = kospi_final_df[["회계년도", "종가"]]
close_df['소속코드'] = 1
close_df = close_df.sort_values(['회계년도'],ignore_index=True)
close_df["인덱스수익률"] = close_df["종가"].pct_change(1) * 100
close_df

,회계년도,종가,소속코드,인덱스수익률
0,2012/12,1997.05,1,NaN
1,2013/12,2011.34,1,0.715555
2,2014/12,1915.59,1,-4.760508
3,2015/12,1961.31,1,2.386732
4,2016/12,2026.46,1,3.321759
5,2017/12,2467.49,1,21.763568
6,2018/12,2041.04,1,-17.282745
7,2019/12,2197.67,1,7.674029
8,2020/12,2873.47,1,30.750750
9,2021/12,2977.65,1,3.625582


In [46]:
close_df.to_excel('/Users/kimboyun/Desktop/Code/UBION_Project2/코스피수익률.xlsx', index=False)

In [34]:
for ticker in stock.get_index_ticker_list(market='KOSDAQ'):
    print(ticker, stock.get_index_ticker_name(ticker))

2001 코스닥
2002 코스닥 대형주
2003 코스닥 중형주
2004 코스닥 소형주
2012 일반서비스
2024 제조
2026 건설
2027 유통
2029 운송·창고
2031 금융
2037 오락·문화
2056 음식료·담배
2058 섬유·의류
2062 종이·목재
2063 출판·매체복제
2065 화학
2066 제약
2067 비금속
2068 금속
2070 기계·장비
2072 전기전자
2074 의료·정밀기기
2075 운송장비·부품
2077 기타제조
2114 통신
2118 IT 서비스
2181 코스닥 우량기업부
2182 코스닥 벤처기업부
2183 코스닥 중견기업부
2184 코스닥 기술성장기업부
2189 코스닥 글로벌
2203 코스닥 150
2212 코스닥 150 소재
2213 코스닥 150 산업재
2214 코스닥 150 필수소비재
2215 코스닥 150 자유소비재
2216 코스닥 150 정보기술
2217 코스닥 150 헬스케어
2218 코스닥 150 커뮤니케이션서비스


In [35]:
def get_previous_trading_day(date):
    """
    입력 날짜가 휴장일이면 다음 영업일까지 반복해서 이동
    date: 'YYYYMMDD' 문자열
    """
    while True:
        # 삼성전자(005930)의 OHLCV 데이터 확인
        df = stock.get_market_ohlcv_by_date(date, date, "005930")
        if not df.empty:
            return date
        # 휴장일이면 이전날로 이동
        dt = datetime.strptime(date, "%Y%m%d") - timedelta(days=+1)
        print(f"{date}는 휴장일입니다. 이전 영업일을 검색합니다...")
        date = dt.strftime("%Y%m%d")

start_year = 2014
end_year = 2024

records = []
for year in range(start_year-1, end_year + 1):
    date = f"{year}1231"  # 4월말 기준 (휴장일일 경우 이후 영업입)
    trading_date = get_previous_trading_day(date)
    df = stock.get_index_ohlcv(trading_date, trading_date, "2001")
    df = df.reset_index(drop=True)
    df['회계년도'] = f"{year}/12"
    records.append(df)

# DataFrame 합치기
final_df = pd.concat(records, ignore_index=True)

20131231는 휴장일입니다. 이전 영업일을 검색합니다...
20141231는 휴장일입니다. 이전 영업일을 검색합니다...
20151231는 휴장일입니다. 이전 영업일을 검색합니다...
20161231는 휴장일입니다. 이전 영업일을 검색합니다...
20161230는 휴장일입니다. 이전 영업일을 검색합니다...
20171231는 휴장일입니다. 이전 영업일을 검색합니다...
20171230는 휴장일입니다. 이전 영업일을 검색합니다...
20171229는 휴장일입니다. 이전 영업일을 검색합니다...
20181231는 휴장일입니다. 이전 영업일을 검색합니다...
20181230는 휴장일입니다. 이전 영업일을 검색합니다...
20181229는 휴장일입니다. 이전 영업일을 검색합니다...
20191231는 휴장일입니다. 이전 영업일을 검색합니다...
20201231는 휴장일입니다. 이전 영업일을 검색합니다...
20211231는 휴장일입니다. 이전 영업일을 검색합니다...
20221231는 휴장일입니다. 이전 영업일을 검색합니다...
20221230는 휴장일입니다. 이전 영업일을 검색합니다...
20231231는 휴장일입니다. 이전 영업일을 검색합니다...
20231230는 휴장일입니다. 이전 영업일을 검색합니다...
20231229는 휴장일입니다. 이전 영업일을 검색합니다...
20241231는 휴장일입니다. 이전 영업일을 검색합니다...


In [36]:
final_df

코스닥,시가,고가,저가,종가,거래량,거래대금,상장시가총액,회계년도
0,498.41,500.05,497.54,499.99,234608083,992814302215,119292529289220,2013/12
1,540.58,543.08,540.28,542.97,388705704,1687681837127,143087772098890,2014/12
2,677.08,683.34,674.87,682.35,512431741,2834865118445,201631307399583,2015/12
3,626.21,631.51,624.23,631.44,582801453,2641713810232,201523388183533,2016/12
4,791.76,798.68,790.34,798.42,1066158680,6670795864435,282740051437771,2017/12
5,670.83,675.89,669.71,675.65,598210555,3033870814294,228238295556450,2018/12
6,662.06,669.83,661.62,669.83,789329743,3799412603271,241350997492726,2019/12
7,957.38,971.04,955.38,968.42,1638729235,16629392847377,385582620024419,2020/12
8,1029.74,1034.87,1021.59,1033.98,907418457,10102608496296,446297029881938,2021/12
9,691.35,693.12,675.80,679.29,952158164,4811065003230,315498720104122,2022/12


In [37]:
df1 = pd.read_excel('/Users/kimboyun/Desktop/Code/UBION_Project2/코스닥12.xlsx')
df1['회계년도'] = '2012/12'
df1

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,상장시가총액,회계년도
0,2012/12/28,496.32,4.24,0.86,493.66,496.32,493.16,329013,1508580,109121943,2012/12


In [38]:
kospi_final_df  = pd.concat([final_df,df1] , join='outer')

In [47]:
close_df = kospi_final_df[["회계년도", "종가"]]
close_df['소속코드'] = 5
close_df = close_df.sort_values(['회계년도'],ignore_index=True)
close_df["인덱스수익률"] = close_df["종가"].pct_change(1) *100
close_df

,회계년도,종가,소속코드,인덱스수익률
0,2012/12,1997.05,5,NaN
1,2013/12,2011.34,5,0.715555
2,2014/12,1915.59,5,-4.760508
3,2015/12,1961.31,5,2.386732
4,2016/12,2026.46,5,3.321759
5,2017/12,2467.49,5,21.763568
6,2018/12,2041.04,5,-17.282745
7,2019/12,2197.67,5,7.674029
8,2020/12,2873.47,5,30.750750
9,2021/12,2977.65,5,3.625582


In [48]:
close_df.to_excel('/Users/kimboyun/Desktop/Code/UBION_Project2/코스닥수익률.xlsx', index=False)

In [207]:
close_df.to_excel("data/kospi200_rtn.xlsx", index=False)